# DB 연결 및 데이터 테스트

이 노트북은 Flask 애플리케이션의 데이터베이스에 연결하고 `RealEstateTransaction` 및 `PublicParking` 모델에서 데이터를 가져옵니다.

In [2]:
from utils import setup_db_context
import pandas as pd
from sqlalchemy import select

# DB 컨텍스트 설정
app, db = setup_db_context()

from myapp.models import RealEstateTransaction, PublicParking

데이터베이스 연결 및 앱 컨텍스트 푸시 완료.


## 1. 부동산 실거래가 (Real Estate Transactions)

In [3]:
# SQLAlchemy를 사용하여 데이터 가져오기
transactions = RealEstateTransaction.query.limit(5).all()
# print(f"가져온 거래 수: {len(transactions)}")

# DataFrame으로 변환
stmt = select(RealEstateTransaction)
df_ret = pd.read_sql(stmt, db.session.connection())

district_df = df_ret

# 각 거래 평단가 계산
def calc_price_per_sqm(df):
    df = df.copy()
    df['price_per_sqm'] = (df['amount'] * 10000) / df['building_area']
    return df

# 지역구 별 년도의 평단가 평균
def calc_yearly_avg_price_by_district(df):
    return (
        df
        .groupby(['district_name', 'reception_year'])
        .agg(avg_price_per_sqm=('price_per_sqm', 'mean'))
        .reset_index()
        .sort_values(['district_name', 'reception_year'])
    )

# 전체 상승률 계산
def calc_total_change_rate_by_district(df):
    total_rate = (
        df
        .groupby('district_name')['avg_price_per_sqm']
        .agg(first='first', last='last')
        .reset_index()
    )

    total_rate['total_change_rate'] = (
        (total_rate['last'] - total_rate['first'])
        / total_rate['first'] * 100
    ).round(2)

    return total_rate

# 순위 출력
def apply_total_change_rank(df):
    df = df.copy()
    df['rank'] = (
        df['total_change_rate']
        .rank(method='dense', ascending=False)
        .astype(int)
    )
    return df.sort_values(['rank', 'district_name'])

# 포맷팅 round(0)으로 소수점 첫 째 자리에서 반올림 | price_per_sqm_format 컬럼 생성
def format_price_column(df):
    df = df.copy()
    df['price_per_sqm_format'] = (
        df['last']
        .round(0)
        .map(lambda x: f"{int(x):,}")
    )
    return df

avg_df = calc_price_per_sqm(district_df)

yoy_avg_df = calc_yearly_avg_price_by_district(avg_df)

district_rate = calc_total_change_rate_by_district(yoy_avg_df)

result = apply_total_change_rank(district_rate)

result = format_price_column(result)



result


,district_name,first,last,total_change_rate,rank,price_per_sqm_format
0,강남구,1.545295e+07,2.418809e+07,56.53,1,"24,188,089"
14,서초구,1.489370e+07,2.287844e+07,53.61,2,"22,878,439"
23,중구,1.031133e+07,1.520279e+07,47.44,3,"15,202,787"
1,강동구,9.327205e+06,1.369928e+07,46.87,4,"13,699,281"
17,송파구,1.157471e+07,1.665364e+07,43.88,5,"16,653,642"
13,서대문구,9.065377e+06,1.294764e+07,42.83,6,"12,947,643"
20,용산구,1.624407e+07,2.287339e+07,40.81,7,"22,873,386"
11,동작구,1.035289e+07,1.444051e+07,39.48,8,"14,440,507"
5,광진구,9.561117e+06,1.331181e+07,39.23,9,"13,311,809"
18,양천구,7.968470e+06,1.099105e+07,37.93,10,"10,991,049"


## 2. 공영주차장 (Public Parkings)

In [4]:
# SQLAlchemy를 사용하여 데이터 가져오기
parkings = PublicParking.query.limit(5).all()
print(f"가져온 주차장 레코드 수: {len(parkings)}")

# DataFrame으로 변환
stmt = select(PublicParking)
df_pp = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_pp)}")
df_pp.head()

가져온 주차장 레코드 수: 5
총 레코드 수: 1875


,pp_id,parking_code,parking_name,address,parking_type,parking_type_name,operation_type,operation_type_name,phone_number,parking_status_available,...,bus_basic_time_min,bus_add_rate,bus_add_time_min,day_max_rate,lat,lng,share_parking_company_name,share_parking,share_parking_company_link,share_parking_etc
0,1,171721,세종로 공영주차장(시),종로구 세종로 80-1,NW,노외 주차장,1,시간제 주차장,02-2290-6566,1,...,0,0,0,30900,37.573403,126.975884,,N,,
1,2,171730,종묘주차장 공영주차장(시),종로구 훈정동 2-0,NW,노외 주차장,1,시간제 주차장,02-2290-6166,1,...,0,0,0,28800,37.571504,126.994969,,N,,
2,3,171900,훈련원공원 공영주차장(시),중구 을지로5가 40-3,NW,노외 주차장,1,시간제 주차장,02-3405-4597,1,...,0,0,0,39600,37.567400,127.003521,,N,,
3,4,172051,한강진역 공영주차장(시),용산구 한남동 728-27,NW,노외 주차장,1,시간제 주차장,02-795-6406,1,...,0,0,0,28800,37.539522,127.002580,,N,,
4,5,172065,용산주차빌딩 공영주차장(시),용산구 한강로2가 12-9,NW,노외 주차장,1,시간제 주차장,02-2290-6014,1,...,0,0,0,21600,37.534364,126.965418,,N,,
